In [5]:
import pandas as pd
import numpy as np
from scipy.stats.stats import pearsonr  
import warnings
warnings.filterwarnings('ignore')

In [6]:
df = pd.read_excel(r'Test.xlsx')
df = df.loc[:,['RTKM_P','RTKM','MTSS','MGTS']]
df = df.pct_change()
df = df.iloc[1:]
list1 = []
for i in range(len(df)):
    list1.append(i)
list1
df.index = list1
n = len(df)
X = df.as_matrix()
X = 100*X
df

,RTKM_P,RTKM,MTSS,MGTS
0,-0.067498,-0.081591,-0.023810,-0.020295
1,0.035405,0.050695,0.056098,0.003766
2,-0.012158,0.010540,0.043880,0.031895
3,-0.007692,0.000232,0.025885,0.001818
4,-0.007752,-0.020971,-0.009058,0.047187
5,0.015625,0.052071,0.030686,-0.024263
6,-0.001538,0.011586,-0.027660,0.001776
7,0.014638,0.034138,0.019110,0.037234
8,0.017464,0.021505,-0.011933,-0.008547
9,0.000000,0.008421,0.011430,-0.008621


In [7]:
class HopfieldNetwork:
    def __init__(self, N, X, _lambda):      
        self.N = N  # number of neurons
        self.w = np.zeros((N, N))  # weight matrix
        self.xi = np.empty((N, 0), dtype="float")  # array with saved patterns
        self.mu = np.empty(N)
        for i in range(N):
            self.mu[i] = np.mean(X[:,i])
        self._lambda = _lambda
        self.b = self._lambda*self.mu
        self.S = np.ones(N, dtype="float")  # state of the neurons
        self.t = 0  # time steps

    def train_pattern(self, input_pattern, X):
        self.w = construct_hebb_matrix(input_pattern, X, self._lambda)
        self.xi = np.column_stack((self.xi, input_pattern))
        
    def update_neurons_with_finite_temp(self, iterations, beta):
        self.t += iterations
        for _ in range(iterations):
            for i in range(self.N): 
                self.S[i] = (
                    0.1 + 
                    (
                        0.3 / (1 + np.exp(-1 * beta * (np.dot(self.w[i],self.S) + (1-self._lambda)*self.mu[i])))
                    )
                    
                )
                
def construct_hebb_matrix(xi, X, _lambda):
    n = xi.shape[0]
    w = np.zeros((n, n))
    corr = np.corrcoef(X.T)
    for i in range(n):
        for j in range(n):
            w[i,j] = -2*_lambda*corr[i][j]

    return w


In [8]:
hopfield_network = HopfieldNetwork(N=4, X=X, _lambda=0.5)
xi = np.array([0.,0.,0.,0.])
hopfield_network.train_pattern(input_pattern=xi, X=X)
timesteps = 10
for t in range(timesteps):
    hopfield_network.update_neurons_with_finite_temp(1, beta=1)

s = hopfield_network.S
s = s/s.sum(axis=0,keepdims=1)

for i in range(len(s)):
    print(f'Stock {df.columns[i]}: {s[i]}')

Stock RTKM_P: 0.23266022983278803
Stock RTKM: 0.23508145333876057
Stock MTSS: 0.2528179159326697
Stock MGTS: 0.27944040089578165
